In [4]:
import cvxpy as cp
import numpy as np
from itertools import product

In [7]:
shipping_cost = np.array([[1,3,2],[3,2,2]]).reshape(2,3)
all_demand = np.array([[100,150,120],[120,180,150],[150,200,180]]).reshape(3,3)
selling_price = np.array([[16,16]]).reshape(2,1)
produce = np.array([100,200]).reshape(2,1)
production_cost = np.array([11,10]).reshape(2,1)

In [8]:
senario = np.array([[100,120,150],
       [150,180,200],[120,150,180]])
senario_comb = []
index = np.arange(0,3)
index_permute = product(index,repeat=3)
for i in index_permute:
    temp_array =[]
    for j in range(0,3):
        temp_array.append(float(senario[j,:][i[j]]))
    senario_comb.append(temp_array)
all_demand = np.array(senario_comb).reshape(27,3)

In [10]:
produce = cp.Variable(shape=(2,1))
sell = cp.Variable(shape=(54,3))
salvage = cp.Variable(shape=(54,1))
produce_limit = np.array([300,300]).reshape(2,1)

constraints_senario = []
for index,demand in enumerate(all_demand):
    i = index*2
    constraints_senario.append(cp.sum(sell[i:i+2,:],axis=1,keepdims=True) +salvage[i:i+2,:] <= produce)
    constraints_senario.append(cp.sum(sell[i:i+2,:],axis=0,keepdims=True) <= demand.reshape(1,-1))
    
   
constraints = [produce>=0,produce<=produce_limit,sell>= np.zeros_like(sell),salvage>= np.zeros_like(salvage)]
constraints.extend(constraints_senario)


objective = cp.Minimize((production_cost.T@produce)-(16/27)*cp.sum(sell,keepdims=True) + (1/27)*cp.trace(((((np.repeat([shipping_cost],27,axis=0).reshape(-1,3))@(sell.T))))))
problem = cp.Problem(objective, constraints)
problem.solve()

-1609.9999997418313

In [11]:
produce.value

array([[120.00000002],
       [299.99999999]])